# Brain tumor 3D segmentation with MONAI

This tutorial uses cached data generated by the Brats 3D Segmentation tutorial from MONAI to demonstrate how to use the TensorBoardPlugin3D.

The dataset comes from http://medicaldecathlon.com/.  
Target: Gliomas segmentation necrotic/active tumour and oedema  
Modality: Multimodal multisite MRI data (FLAIR, T1w, T1gd,T2w)  
Size: 750 4D volumes (484 Training + 266 Testing)  
Source: BRATS 2016 and 2017 datasets.  
Challenge: Complex and heterogeneously-located targets

Below figure shows image patches with the tumor sub-regions that are annotated in the different modalities (top left) and the final labels for the whole dataset (right).
(Figure taken from the [BraTS IEEE TMI paper](https://ieeexplore.ieee.org/document/6975210/))

![image](https://github.com/Project-MONAI/tutorials/blob/master/figures/brats_tasks.png?raw=1)

The image patches show from left to right:
1. the whole tumor (yellow) visible in T2-FLAIR (Fig.A).
1. the tumor core (red) visible in T2 (Fig.B).
1. the enhancing tumor structures (light blue) visible in T1Gd, surrounding the cystic/necrotic components of the core (green) (Fig. C).
1. The segmentations are combined to generate the final labels of the tumor sub-regions (Fig.D): edema (yellow), non-enhancing solid core (red), necrotic/cystic core (green), enhancing core (blue).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KitwareMedical/tensorboard-plugin-3d/blob/main/demo/notebook/cached_brats_segmentation_3d.ipynb)

In [ ]:
!pip install -q "monai-weekly[nibabel, tqdm, ignite]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline
!pip install -q tensorboard-plugin-3d

In [ ]:
import tempfile
import shutil
import os
import gdown

## Setup data directory

You can specify a directory to download the data to with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads. If not specified a temporary directory will be used.

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
url = 'https://drive.google.com/drive/folders/1Jwqv--tom2T2B0WIPEYVDfq8dpMkSdTJ?usp=sharing'
gdown.download_folder(url, output=root_dir, quiet=True)
print(root_dir)

## Check best model output with the input image and label

In our sample data we have four folders to select from: `all_data`, `input_with_label`, `output_with_label`, and `output_only`. All options contain data for the first three items in out DataLoader.
- `output_only` contains the output model only.
- `output_with_label` contains the output model with the input label so that input and output can be viewed at the same time and easily compared.
- `input_with_label` contains the original input image and label so that you can view the original data with the transforms applied
- `all_data` contains the data from all three other folders: `input_with_label`, `output_with_label`, and `output_only`. This allows us to view the transformed input image and label followed by the output model by itself, then finally the output model with the input label for each of the three options.

In [ ]:
output_only = os.path.join(root_dir, "output_only")
output_with_label = os.path.join(root_dir, "output_with_label")
input_with_label = os.path.join(root_dir, "input_with_label")
all_data = os.path.join(root_dir, "all_data")

Lets look at `all_data` to get an idea of what each sample output looks like in the TensorBoard plugin. Setting the `--logdir` flag will tell TensorBoard which directory to look at for data.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=$all_data

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)